# This notebook is used to create Embedding and assign it as property to Movie nodes
## Create Vector Index

## Seach any movie title using Vector Search

In [1]:
!pip install neo4j -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 5.9 MB/s eta 0:00:00


In [2]:
uri="neo4j+s://8e403408.databases.neo4j.io"  # Get from Neo4j Aura console
username = "neo4j"  # usually 'neo4j'
password = "adswC-DiHRpM6BAnMbyHjoIIqEyrZUS2VsKcJdJ7LY4"  # set during DB setup

In [7]:
# Create the driver
from neo4j import GraphDatabase
import csv

driver = GraphDatabase.driver(uri, auth=(username, password))

In [4]:
def set_plot_embedding(tx, movieId, embedding):
    tx.run(
        """
        MATCH (m:Movie {movieId: $movieId})
        CALL db.create.setNodeVectorProperty(m, 'plotEmbedding', apoc.convert.fromJsonList($embedding))
        RETURN m
        """,
        movieId=movieId,
        embedding=embedding
    )

In [5]:
# Read CSV and push embeddings
with driver.session() as session:
  with open("openai-embeddings.csv", newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      #print(row)
      #movieId = int(row['movieId']) if row['movieId'].isdigit() else row['movieId']
      movieId = row['movieId']
      embedding = row['embedding']
      session.execute_write(set_plot_embedding, movieId, embedding)

print("Embedding vectors updated in Neo4j.")
driver.close()

Embedding vectors updated in Neo4j.


In [8]:
#Creating Vector Index on the Movie label against the property=plotEmbedding
with driver.session() as session:
  session.execute_write(lambda tx: tx.run(
        """
        CREATE VECTOR INDEX moviePlots IF NOT EXISTS
        FOR (m:Movie)
        ON m.plotEmbedding
        OPTIONS {indexConfig: {
        `vector.dimensions`: 1536,
        `vector.similarity_function`: 'cosine'
        }}
        """
    )
  )

In [9]:
# Query the nodes using Vector Search

def find_similar_movies(tx, movie_title, index_name="moviePlots", top_k=6):
    result = tx.run(
        """
        MATCH (m:Movie {title: $movie_title})
        CALL db.index.vector.queryNodes($index_name, $top_k, m.plotEmbedding)
        YIELD node, score
        RETURN node.title AS title, node.plot AS plot, score
        """,
        movie_title=movie_title,
        index_name=index_name,
        top_k=top_k
    )
    return [record.data() for record in result]


In [10]:
with driver.session() as session:
  similar_movies = session.execute_read(find_similar_movies, "Toy Story")
  for movie in similar_movies:
      print(f"Title: {movie['title']}, Score: {movie['score']:.2f}")

Title: Toy Story, Score: 1.00
Title: Indian in the Cupboard, The, Score: 0.92
Title: Powder, Score: 0.91
Title: Jumanji, Score: 0.91
Title: Babe, Score: 0.90
Title: Get Shorty, Score: 0.90
